In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%%time
import re
import random
%run ../py/sql_utlis.py
%run ../py/html_analysis.py

ZMQInteractiveShell_obj = get_ipython()
su = SqlUtilities()
_, CURSOR = su.get_jh_conn_cursor()
ha = HeaderAnalysis()
HEADER_PATTERN_DICT = s.load_object('HEADER_PATTERN_DICT')

Wall time: 16.8 s


In [3]:

# SET other subtypes as 0; assume no rows affected if primary and secondary columns are the same
subtypes_list = ['is_task_scope', 'is_minimum_qualification', 'is_preferred_qualification',
                 'is_legal_notification', 'is_job_title', 'is_office_location', 'is_job_duration',
                 'is_supplemental_pay', 'is_educational_requirement', 'is_interview_procedure',
                 'is_corporate_scope', 'is_posting_date', 'is_other']
for primary_column in subtypes_list:
    for secondary_column in subtypes_list:
        CURSOR.execute(su.set_secondary_column0_formatted_sql_str.format(secondary_column, primary_column,
                                                                         secondary_column))

In [4]:

child_strs_set = set()
for file_name, feature_dict_list in HEADER_PATTERN_DICT.items():
    for feature_dict in feature_dict_list:
        if 'is_task_scope' not in feature_dict:
            child_strs_set.add(feature_dict['child_str'])
CHILD_STRS_LIST = list(child_strs_set)
SQL_STR = '''
    SELECT np.[navigable_parent]
    FROM [Jobhunting].[dbo].[NavigableParents] np
    WHERE
        (np.[is_task_scope] IS NULL) OR
        (np.[is_minimum_qualification] IS NULL) OR
        (np.[is_preferred_qualification] IS NULL) OR
        (np.[is_legal_notification] IS NULL) OR
        (np.[is_job_title] IS NULL) OR
        (np.[is_office_location] IS NULL) OR
        (np.[is_job_duration] IS NULL) OR
        (np.[is_supplemental_pay] IS NULL) OR
        (np.[is_educational_requirement] IS NULL) OR
        (np.[is_interview_procedure] IS NULL) OR
        (np.[is_corporate_scope] IS NULL) OR
        (np.[is_posting_date] IS NULL) OR
        (np.[is_other] IS NULL);'''

In [5]:

def get_dictionary_code():
    nonheader_quals_df = pd.DataFrame(su.get_execution_results(CURSOR, SQL_STR, verbose=False))
    mask_series = nonheader_quals_df.navigable_parent.isin(CHILD_STRS_LIST)
    output_str = ''
    child_strs_list = nonheader_quals_df[mask_series].navigable_parent.tolist()
    tag_str = random.choice(child_strs_list)
    output_str += f'\n# Only {len(child_strs_list):,} records to go\n'
    if "'" in tag_str:
        tag_str = tag_str.replace('"', '\\"')
        output_str += f'tag_str = "{tag_str}"\n'
    else:
        output_str += f"tag_str = '{tag_str}'\n"
    output_str += 'sql_str = r"""UPDATE NavigableParents\n    SET\n        is_header = 0,\n'
    output_str += '        is_task_scope = 0,\n        is_minimum_qualification = 0,\n'
    output_str += '        is_preferred_qualification = 0,\n        is_legal_notification = 0,\n'
    output_str += '        is_job_title = 0,\n        is_office_location = 0,\n        is_job_duration = 0,\n'
    output_str += '        is_supplemental_pay = 0,\n        is_educational_requirement = 0,\n'
    output_str += '        is_interview_procedure = 0,\n        is_corporate_scope = 0,\n'
    output_str += '        is_posting_date = 0,\n        is_other = 0\n'
    output_str += '    WHERE (navigable_parent LIKE ? ESCAPE \'\\\')"""\n'
    output_str += r"cursor_obj = CURSOR.execute(sql_str, (su.wc_rgx.sub(r'\\\g<1>', tag_str),))"
    output_str += '\nCURSOR.commit()'
    
    return output_str

In [ ]:

ZMQInteractiveShell_obj.set_next_input(text=get_dictionary_code(), replace=True)